# 0. Mess around space

In [1]:
import csv
from impt_functions import *

In [2]:
dt_dict_overall = {'Run1': [0.8, 0.9], 'Run2': [3.0, 2], 'Run3': [10, 11]}
save_dict_2_csv(dt_dict_overall, 'dt_overall_results.csv')

# 1. Imports and Set Up

In [3]:
from impt_functions import *
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import pandas as pd 
import numpy as np
from fairlearn.reductions import ExponentiatedGradient, GridSearch, DemographicParity, EqualizedOdds, \
    TruePositiveRateParity, FalsePositiveRateParity, ErrorRateParity, BoundedGroupLoss
from fairlearn.metrics import *
from raiwidgets import FairnessDashboard
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
%matplotlib inline

In [4]:
# Dict used for saving overall results to a csv later
# Example: overall = {'Unmitigated':[], 'EG DP': [], 'EG EO': [],
#                   'EG EOO': [], 'EG FPRP': [], 'EG ERP': [],
#                   'GS DP': [], 'GS EO': [],
#                   'GS EOO': [], 'GS FPRP': [], 'GS ERP': []}
# Example: black_results = {'Unmitigated Black': [], 
#                          'EG DP Black': [], ..., 'GS ERP Black': []}

# Instantiate lists for holding results
unmitigated, unmitigated_black, unmitigated_white = [], [], []

eg_dp, eg_dp_black, eg_dp_white = [], [], []
eg_eo, eg_eo_black, eg_eo_white = [], [], []
eg_eoo, eg_eoo_black, eg_eoo_white = [], [], []
eg_fprp, eg_fprp_black, eg_fprp_white = [], [], []
eg_erp, eg_erp_black, eg_erp_white = [], [], []

gs_dp, gs_dp_black, gs_dp_white = [], [], []
gs_eo, gs_eo_black, gs_eo_white = [], [], []
gs_eoo, gs_eoo_black, gs_eoo_white = [], [], []
gs_fprp, gs_fprp_black, gs_fprp_white = [], [], []
gs_erp, gs_erp_black, gs_erp_white = [], [], []

# 2. Prepare data

In [5]:
data = get_data('/home/mackenzie/git_repositories/delayedimpact/data/simData_oom100.csv')

       score  repay_probability  race  repay_indices
0        601              75.21     1              1
1        693              95.15     1              1
2        791              98.62     1              1
3        637              86.69     1              1
4        775              98.45     1              1
...      ...                ...   ...            ...
99995    797              98.73     1              1
99996    562              58.57     1              1
99997    687              94.60     1              1
99998    589              70.61     1              1
99999    555              52.97     1              0

[100000 rows x 4 columns]


In [6]:
X_train, X_test, y_train, y_test, race_train, race_test, sample_weight_train, sample_weight_test = prep_data(data=data, test_size=0.3, weight_index=1)

Here are the x values:  [[601   1]
 [693   1]
 [791   1]
 ...
 [687   1]
 [589   1]
 [555   1]] 

Here are the y values:  [1 1 1 ... 1 1 0]
Sample weights are all equal.


# Decision Tree Classifier

## Train DTC classifier + Collect Predictions
NOTE: atm sample_weight are all 1s

In [7]:
# Reference: https://www.datacamp.com/community/tutorials/decision-tree-classification-python

# Initialize classifier:
clf = DecisionTreeClassifier()

# Train the classifier:
model = clf.fit(X_train,y_train)

# Make predictions with the classifier:
y_predict = model.predict(X_test)

# Scores on test set
test_scores = model.predict_proba(X_test)[:, 1]

### Evaluation of classifier overall

In [8]:
# Metrics
models_dict = {"Unmitigated": (y_predict, test_scores)}
get_metrics_df(models_dict, y_test, race_test)

,Unmitigated
Overall selection rate,0.729567
Demographic parity difference,0.450625
Demographic parity ratio,0.424364
------,
Overall balanced error rate,0.151732
Balanced error rate difference,0.0170221
------,
True positive rate difference,0.190934
True negative rate difference,0.156889
False positive rate difference,0.156889


In [9]:
cm = confusion_matrix(y_test, y_predict)
print(cm)
results_dict = classification_report(y_test, y_predict, output_dict=True)
print(classification_report(y_test, y_predict))
# Add accuracy to the results list
unmitigated.append(round(results_dict['accuracy']*100, 2))
f1_micro, f1_weighted, f1_binary = get_f1_scores(y_test, y_predict)
f1_str = str(round(f1_micro*100, 2))+"/"+str(round(f1_weighted*100, 2))+"/"+str(round(f1_binary*100, 2))
# Add f1 scores to results list
unmitigated.append(f1_str)
# Add Selection rate to results list
sr = get_selection_rates(y_test, y_predict, race_test, 0)
unmitigated.append(round(sr*100, 2))
# Add Outcome rates to results list
tnr, tpr, fner, fper = evaluation_outcome_rates(y_test, y_predict, sample_weight_test)
unmitigated.append(round(tnr*100, 2))
unmitigated.append(round(tpr*100, 2))
unmitigated.append(round(fner*100, 2))
unmitigated.append(round(fper*100, 2))

[[ 6644  2036]
 [ 1469 19851]]
              precision    recall  f1-score   support

           0       0.82      0.77      0.79      8680
           1       0.91      0.93      0.92     21320

    accuracy                           0.88     30000
   macro avg       0.86      0.85      0.86     30000
weighted avg       0.88      0.88      0.88     30000

F1 score micro: 
0.8831666666666667
F1 score weighted: 
0.8819608768731572
F1 score binary: 
0.9188788853657973



The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


Selection Rate Overall:  0.7295666666666667
TNR=TN/(TN+FP)=  0.7654377880184332
TPR=TP/(FP+FN)=  0.9310975609756098
FNER=FN/(FN+TP)=  0.06890243902439025
FPER=FP/(FP+TN)=  0.23456221198156682


### Delayed impact calculated

In [10]:
di_black, di_white = calculate_delayed_impact(X_test, y_test, y_predict, race_test)
# Add DI to results list
di_str = str(round(di_black, 2))+"/"+str(round(di_white, 2))
unmitigated.append(di_str)

The delayed impact of the black group is:  6.958544839255499
The delayed impact of the white group is:  43.80244953504196


### Fairness Metric Evaluation of classifier

In [11]:
dp_diff, eod_diff = print_fairness_metrics(y_test, y_predict, race_test)
# Add Demographic Parity and Equalized Odds Differences to results list
unmitigated.append(round(dp_diff*100, 2))
unmitigated.append(round(eod_diff*100, 2))

DP Difference:  0.45062527207781544
-->difference of 0 means that all groups have the same selection rate
DP Ratio: 0.42436424030390024
-->ratio of 1 means that all groups have the same selection rate 

EOD Difference:  0.1909337288334525
-->difference of 0 means that all groups have the same TN, TN, FP, and FN rates
EOD Ratio: 0.4338376595335293
-->ratio of 1 means that all groups have the same TN, TN, FP, and FN rates rates 



In [12]:
tprd = true_positive_rate_difference(y_test, y_predict, sensitive_features=race_test)
unmitigated.append(round(tprd*100, 2))
print('True Positive Rate Difference: ', tprd)
tnrd = true_negative_rate_difference(y_test, y_predict, sensitive_features=race_test)
unmitigated.append(round(tnrd*100, 2))
print('True Negative Rate Difference: ', tnrd)
fprd = false_positive_rate_difference(y_test, y_predict, sensitive_features=race_test)
unmitigated.append(round(fprd*100, 2))
print('False Positive Rate Difference: ', fprd)
fnrd = false_negative_rate_difference(y_test, y_predict, sensitive_features=race_test)
unmitigated.append(round(fnrd*100, 2))
print('False Negative Rate Difference: ', fnrd)

True Positive Rate Difference:  0.1909337288334525
True Negative Rate Difference:  0.1568894376917046
False Positive Rate Difference:  0.15688943769170458
False Negative Rate Difference:  0.19093372883345247


### Evaluation of classifier by race

In [13]:
results_black, results_white = evaluation_by_race(X_test, y_test, race_test, y_predict, sample_weight_test)
unmitigated_black.extend(results_black)
unmitigated_white.extend(results_white)

EVALUATION FOR BLACK GROUP
[[2071  283]
 [ 297  895]]
              precision    recall  f1-score   support

           0       0.87      0.88      0.88      2354
           1       0.76      0.75      0.76      1192

    accuracy                           0.84      3546
   macro avg       0.82      0.82      0.82      3546
weighted avg       0.84      0.84      0.84      3546

F1 score micro: 
0.8364354201917654
F1 score weighted: 
0.8361947899037017
F1 score binary: 
0.7552742616033755

TNR=TN/(TN+FP)=  0.8797790994052677
TPR=TP/(FP+FN)=  0.7508389261744967
FNER=FN/(FN+TP)=  0.24916107382550334
FPER=FP/(FP+TN)=  0.12022090059473237

EVALUATION FOR WHITE GROUP
[[ 4573  1753]
 [ 1172 18956]]
              precision    recall  f1-score   support

           0       0.80      0.72      0.76      6326
           1       0.92      0.94      0.93     20128

    accuracy                           0.89     26454
   macro avg       0.86      0.83      0.84     26454
weighted avg       0.89    

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


The delayed impact of the black group is:  6.958544839255499
The delayed impact of the white group is:  43.80244953504196


### Save results to dictionaries

In [14]:
unmitigated_key = 'Unmitigated DT'
overall_results_dict = add_values_in_dict({}, unmitigated_key, unmitigated)
black_results_dict = add_values_in_dict({}, unmitigated_key, unmitigated_black)
white_results_dict = add_values_in_dict({}, unmitigated_key, unmitigated_white)
# To use below!!
#save_dict_2_csv(overall_results_dict, 'DT Overall Results')
#save_dict_2_csv(byrace_results_dict, 'DT By Race Results')

In [15]:
print(overall_results_dict)
print(black_results_dict)
print(white_results_dict)

{'Unmitigated DT': [88.32, '88.32/88.2/91.89', 72.96, 76.54, 93.11, 6.89, 23.46, '6.96/43.8', 45.06, 19.09, 19.09, 15.69, 15.69, 19.09]}
{'Unmitigated DT': [83.64, '83.64/83.62/75.53', 33.22, 87.98, 24.92, 12.02, 6.96]}
{'Unmitigated DT': [88.94, '88.94/88.76/92.84', 78.28, 72.29, 5.82, 27.71, 43.8]}


## Exponentiated Gradient Reduction Alg for Adding Fairness Constraints

### Demographic Parity

In [ ]:
eg_dp, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'DP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### Equalized Odds

In [ ]:
eg_eo, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'EO', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### EOO (True Positive Rate Parity)

In [ ]:
eg_tprp, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'TPRP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### False Positive Rate Parity

In [ ]:
eg_fprp, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'FPRP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### Error Rate Parity

In [ ]:
eg_erp, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'ERP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

### Bounded Group Loss (issue, need to figure out loss parameter)

In [ ]:
#eg_bgl, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'BGL', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

## Grid Search Reduction Alg for Adding Fairness Constraints

### Demographic Parity

In [ ]:
gs_dp, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'DP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_dp.lambda_vecs_
print(lambda_vecs[0])

The next few cells come from: https://github.com/fairlearn/fairlearn/blob/main/notebooks/Binary%20Classification%20with%20the%20UCI%20Credit-card%20Default%20Dataset.ipynb

Note: we train multiple models corresponding to different trade-off points between the performance metric (balanced accuracy) and fairness metric.

In [ ]:
grid_search_show(gs_dp, demographic_parity_difference, y_predict, X_test, y_test, race_test, 'DemParityDifference','GS DPD', models_dict, 0.3)

In [ ]:
models_dict.pop('GS DPD')
models_dict

### Equalized Odds Used

In [ ]:
gs_eo, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'EO', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_eo.lambda_vecs_
print(lambda_vecs[0])

In [ ]:
grid_search_show(gs_eo, equalized_odds_difference, y_predict, X_test, y_test, race_test, 'EOddsDifference','GS EO', models_dict, 0.3)

In [ ]:
models_dict.pop('GS EO')
models_dict

### EOO (True Positive Rate Parity)

In [ ]:
gs_tprp, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'TPRP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_tprp.lambda_vecs_
print(lambda_vecs[0])

In [ ]:
grid_search_show(gs_tprp, true_positive_rate_difference, y_predict, X_test, y_test, race_test, 'TPRPDifference','GS TPRP', models_dict, 0.3)

In [ ]:
models_dict.pop('GS TPRP')
models_dict

### False Positive Rate Parity

In [ ]:
gs_fprp, tnr, tpr, fner, fper, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'FPRP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_fprp.lambda_vecs_
print(lambda_vecs[0])

In [ ]:
# NOTE: the below models are the same!!

In [ ]:
grid_search_show(gs_fprp, false_positive_rate_difference, y_predict, X_test, y_test, race_test, 'FPRPDifference','GS FPRP', models_dict, 0.4)

In [ ]:
models_dict.pop('GS FPRP')
models_dict

### Error Rate Parity

In [ ]:
gs_erp, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'ERP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
lambda_vecs = gs_erp.lambda_vecs_
print(lambda_vecs[0])

In [ ]:
# Fairlearn doesnt have an erp difference metric for the below
#grid_search_show(gs_erp, error_difference, y_predict, X_test, y_test, race_test, 'ERDifference','GS ERP', models_dict, 0.3)
#models_dict.pop('GS FPRP')
#models_dict

### Bounded Group Loss (issue, need to figure out loss parameter)

In [ ]:
#gs_bgl, tnr, tpr, fner, fper, sr_overall, di_black, di_white, dp_diff, eod_diff = add_constraint(model, 'BGL', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

In [ ]:
# We can examine the values of lambda_i chosen for us:
#lambda_vecs = gs_dp.lambda_vecs_
#print(lambda_vecs[0])